<a href="https://colab.research.google.com/github/karan747/MLstuff/blob/MLTryouts/CNNTunerModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install keras-tuner


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 50.4 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [13]:
import keras_tuner

In [4]:
data = keras.datasets.fashion_mnist

In [5]:
(train_images,train_lables),(test_images,test_lables) = data.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [12]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [14]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model


In [17]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [19]:
tuner_search = RandomSearch(build_model,objective = "val_accuracy", max_trials = 5 , directory = "output", project_name = "Fashion")

In [21]:
tuner_search.search(train_images, train_lables, epochs = 5, validation_split = 0.2)

Trial 5 Complete [00h 00m 36s]
val_accuracy: 0.8835833072662354

Best val_accuracy So Far: 0.9139166474342346
Total elapsed time: 00h 03m 31s


In [22]:
model = tuner_search.get_best_models(num_models = 1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        32288     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 32)                589856    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 623,594
Trainable params: 623,594
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_images, train_lables, epochs = 12, validation_split=0.2, initial_epoch = 5)

Epoch 6/12
1500/1500 [==============================] - 7s 5ms/step - loss: 0.1437 - accuracy: 0.9473 - val_loss: 0.2869 - val_accuracy: 0.9115
Epoch 7/12
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1112 - accuracy: 0.9589 - val_loss: 0.3059 - val_accuracy: 0.9089
Epoch 8/12
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0870 - accuracy: 0.9682 - val_loss: 0.3369 - val_accuracy: 0.9072
Epoch 9/12
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0680 - accuracy: 0.9746 - val_loss: 0.3709 - val_accuracy: 0.9094
Epoch 10/12
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0514 - accuracy: 0.9816 - val_loss: 0.3995 - val_accuracy: 0.9105
Epoch 11/12
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0454 - accuracy: 0.9840 - val_loss: 0.4642 - val_accuracy: 0.9077
Epoch 12/12
1500/1500 [==============================] - 7s 5ms/step - loss: 0.0361 - accuracy: 0.9864 - val_loss: 0.5358 - val_accura